In [200]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

In [201]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [202]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [203]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [204]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [205]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [206]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=pd.rolling_mean(df_dict[sku]['Sales'], window=7).shift(1) 

In [207]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [208]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [209]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [210]:
cv=2

RF=RandomForestRegressor();
rf_models=[];
for sku in list(df_dict.keys()):
    rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) )
#

GB=ensemble.GradientBoostingRegressor(n_estimators= 500,max_depth= 40, min_samples_split=20,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
for sku in list(df_dict.keys()):
    gb_models.append(cross_val_score(GB,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
#    

XG=xgb.XGBRegressor( #learning_rate =0.1, n_estimators=1000, max_depth=10,
        #min_child_weight=1,gamma=0, subsample=0.8,colsample_bytree=0.8,nthread=4,scale_pos_weight=1,seed=27  
)
xg_models=[]
for sku in list(df_dict.keys()):
    xg_models.append(cross_val_score(XG,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [211]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X[1].shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=220,
                               batch_size=5,
                               verbose=0)

In [212]:
mlp_models=[]
for sku in list(df_dict.keys()):
    mlp_models.append(cross_val_score(MLP,X[sku],y[sku],cv=cv).mean().round(0)  )

In [213]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae.loc['mean'] = mae.mean()
mae['range']=mae.max(axis=1)-mae.min(axis=1)
mae

,MLP,RF,GB,XG,Best,range
1,2202.0,382.0,379.0,374.0,XG,1828.0
2,441.0,415.0,429.0,398.0,XG,43.0
3,758.0,609.0,651.0,597.0,XG,161.0
4,750.0,789.0,790.0,792.0,MLP,42.0
5,397.0,388.0,401.0,400.0,RF,13.0
6,692.0,563.0,626.0,606.0,RF,129.0
7,763.0,781.0,755.0,763.0,GB,26.0
8,664.0,508.0,503.0,558.0,GB,161.0
9,613.0,649.0,683.0,751.0,MLP,138.0
10,388.0,402.0,411.0,421.0,MLP,33.0


In [214]:
#Need lots of rank and counts of how many times models came in 1st  vs 2nd.
def rank_model(df,rank):
    coln = 'Best' + str(rank) 
    sortID = np.argpartition(df[['MLP','RF','GB','XG']].values,rank,axis=1)[:,rank]
    df[coln] = df.columns[sortID]
    

In [215]:
rank_model(mae,0)
rank_model(mae,1)
mae

,MLP,RF,GB,XG,Best,range,Best0,Best1
1,2202.0,382.0,379.0,374.0,XG,1828.0,XG,GB
2,441.0,415.0,429.0,398.0,XG,43.0,XG,RF
3,758.0,609.0,651.0,597.0,XG,161.0,XG,RF
4,750.0,789.0,790.0,792.0,MLP,42.0,MLP,RF
5,397.0,388.0,401.0,400.0,RF,13.0,RF,MLP
6,692.0,563.0,626.0,606.0,RF,129.0,RF,XG
7,763.0,781.0,755.0,763.0,GB,26.0,GB,MLP
8,664.0,508.0,503.0,558.0,GB,161.0,GB,RF
9,613.0,649.0,683.0,751.0,MLP,138.0,MLP,RF
10,388.0,402.0,411.0,421.0,MLP,33.0,MLP,RF


In [216]:
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return pred

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [217]:
#NOTE, because we are retraining models here, a bad initiatlization could lead to different results. 

def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":     
            d[sku]=RF_final(X[sku],y[sku])
                    
        elif scores.loc[sku]["Best"]=="MLP":
            d[sku]=MLP_final(X[sku],y[sku])                 
            
        elif scores.loc[sku]["Best"]=="GB":
            d[sku]=GB_final(X[sku],y[sku])
            
        elif scores.loc[sku]["Best"]=="XG":
            d[sku]=XG_final(X[sku],y[sku])
      
    return d     

In [218]:
bm_dict=best_model(mae,X,y)

In [219]:
#MAE of Best Model forecasting over all 935 days.  NN do very poorly here!
mae2=[]
for sku in list(df_dict.keys()):
    mae2.append( mean_absolute_error(y[sku], bm_dict[sku])   )
mae2=pd.DataFrame(mae2)
mae2.index = mae2.index + 1
mae2.round(1)

,0
1,260.8
2,286.1
3,438.0
4,562.4
5,143.3
6,151.5
7,87.9
8,50.2
9,373.6
10,304.9


In [220]:
sku_pred=pd.concat([ bm_dict[1],bm_dict[2],bm_dict[3],bm_dict[4],bm_dict[5],bm_dict[6],bm_dict[7],bm_dict[8],bm_dict[9],bm_dict[10],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred.shape

(935, 11)